In [1]:
from numpy import pi

from oqd_compiler_infrastructure import PrettyPrint, Post, FixedPoint, Chain

from oqd_core.compiler.math.passes import simplify_math_expr, evaluate_math_expr
from oqd_core.compiler.math.rules import (
    DistributeMathExpr,
    ProperOrderMathExpr,
    PartitionMathExpr,
    PrintMathExpr,
)

from trical.light_matter2 import *

In [2]:
printer = Post(OperatorPrinter())

In [3]:
op = Displacement(alpha=WaveCoefficient(amplitude=1, frequency=0, phase=0))

print(printer(op))

class_='Displacement' alpha=WaveCoefficient(class_='WaveCoefficient', amplitude=MathNum(class_='MathNum', value=1), frequency=MathNum(class_='MathNum', value=0), phase=MathNum(class_='MathNum', value=0))


In [10]:
downstate = Level(
    principal=6,
    spin=1 / 2,
    orbital=0,
    nuclear=1 / 2,
    spin_orbital=1 / 2,
    spin_orbital_nuclear=0,
    spin_orbital_nuclear_magnetization=0,
    energy=0,
)
upstate = Level(
    principal=6,
    spin=1 / 2,
    orbital=0,
    nuclear=1 / 2,
    spin_orbital=1 / 2,
    spin_orbital_nuclear=1,
    spin_orbital_nuclear_magnetization=0,
    energy=2 * pi * 12.643e9,
)
estate = Level(
    principal=5,
    spin=1 / 2,
    orbital=1,
    nuclear=1 / 2,
    spin_orbital=1 / 2,
    spin_orbital_nuclear=0,
    spin_orbital_nuclear_magnetization=0,
    energy=2 * pi * 811.52e12,
)
estate2 = Level(
    principal=5,
    spin=1 / 2,
    orbital=1,
    nuclear=1 / 2,
    spin_orbital=1 / 2,
    spin_orbital_nuclear=1,
    spin_orbital_nuclear_magnetization=0,
    energy=2 * pi * 811.52e12,
)

transitions = [
    Transition(
        level1=downstate,
        level2=estate2,
        einsteinA=1.23e8,
        multipole="E1",
    ),
    Transition(
        level1=upstate,
        level2=estate,
        einsteinA=1.23e8,
        multipole="E1",
    ),
]

Yb171 = Ion(
    mass=171,
    charge=1,
    position=[0, 0, 0],
    levels=[downstate, upstate, estate, estate2],
    transitions=transitions,
)

COM_x = Phonon(energy=2 * pi * 5e6, eigenvector=[1, 0, 0])
COM_y = Phonon(energy=2 * pi * 5e6, eigenvector=[0, 1, 0])
COM_z = Phonon(energy=2 * pi * 1e6, eigenvector=[0, 0, 1])

system = System(
    ions=[
        Yb171,
    ],
    modes=[
        COM_x,
        COM_y,
        COM_z,
    ],
)

beam = Beam(
    transition=transitions[1],
    rabi=2 * pi * 1e9,
    detuning=0,
    phase=0,
    polarization=[0, 0, 1],
    wavevector=[0, 2 * pi / 0.02371213, 0],
    target=0,
)

beam2 = Beam(
    transition=transitions[0],
    rabi=2 * pi * 1e9,
    detuning=0,
    phase=0,
    polarization=[0, 0, 1],
    wavevector=[0, 2 * pi / 0.02371213, 0],
    target=0,
)

protocol = ParallelProtocol(
    sequence=[
        Pulse(beam=beam, duration=1),
        Pulse(beam=beam2, duration=1),
    ]
)

circuit = AtomicCircuit(system=system, protocol=protocol)

In [11]:
compiler = Post(ConstructHamiltonian())

op = compiler(circuit)

print(printer(op))

Zero + (Zero + (0.0) * |0><0| + (79438311838.67151) * |1><1| + (5098930540482378.0) * |2><2| + (5098930540482378.0) * |3><3|) @ I @ I @ I + I @ (31415926.535897933) * C * A @ I @ I + I @ I @ (31415926.535897933) * C * A @ I + I @ I @ I @ (6283185.307179586) * C * A + (Zero + (5.344929169906417e-11 * (2 * 0.0013272093648958553 * (1.054571817e-34 * 6283185307.179586 / 8.563720752160326e-30) ** 2 / 0.0026544187297917105) ** 0.5 * 1.602176634e-19 / 1.054571817e-34 / 2 * exp(1j * (5098851102170539.0 + 0 * t)) + 5.344929169906417e-11 * (2 * 0.0013272093648958553 * (1.054571817e-34 * 6283185307.179586 / 8.563720752160326e-30) ** 2 / 0.0026544187297917105) ** 0.5 * 1.602176634e-19 / 1.054571817e-34 / 2 * exp(1j * (-1 * (5098851102170539.0 + 0) * t + -1 * 0))) * |0><3| + |3><0| + (5.3450540785756624e-11 * (2 * 0.0013272093648958553 * (1.054571817e-34 * 6283185307.179586 / 8.563720752160326e-30) ** 2 / 0.0026544187297917105) ** 0.5 * 1.602176634e-19 / 1.054571817e-34 / 2 * exp(1j * (509885110217

In [12]:
from oqd_compiler_infrastructure import RewriteRule
from oqd_core.interface.math import MathNum


class Prune(RewriteRule):
    def map_OperatorAdd(self, model):
        if isinstance(model.op1, Zero):
            return model.op2
        if isinstance(model.op2, Zero):
            return model.op1

    def map_OperatorMul(self, model):
        if isinstance(model.op1, Zero) or isinstance(model.op2, Zero):
            return Zero()

    def map_OperatorScalarMul(self, model):
        if isinstance(
            model.coeff, WaveCoefficient
        ) and model.coeff.amplitude == MathNum(value=0):
            return Zero()

    def map_CoefficientAdd(self, model):
        if isinstance(
            model.coeff1, WaveCoefficient
        ) and model.coeff1.amplitude == MathNum(value=0):
            return model.coeff2
        if isinstance(
            model.coeff2, WaveCoefficient
        ) and model.coeff2.amplitude == MathNum(value=0):
            return model.coeff1

    def map_Wave(self, model):
        if isinstance(
            model.lamb_dicke, WaveCoefficient
        ) and model.lamb_dicke.amplitude == MathNum(value=0):
            return Identity()


simplify = Chain(
    Chain(
        FixedPoint(Post(DistributeMathExpr())),
        FixedPoint(Post(ProperOrderMathExpr())),
        FixedPoint(Post(PartitionMathExpr())),
    ),
    simplify_math_expr,
    FixedPoint(Post(Prune())),
    Chain(
        FixedPoint(Post(DistributeMathExpr())),
        FixedPoint(Post(ProperOrderMathExpr())),
        FixedPoint(Post(PartitionMathExpr())),
    ),
    simplify_math_expr,
)

simplified_op = simplify(op)

print(printer(simplified_op))

((79438311838.67151) * |1><1| + (5098930540482378.0) * |2><2| + (5098930540482378.0) * |3><3|) @ I @ I @ I + I @ (31415926.535897933) * C * A @ I @ I + I @ I @ (31415926.535897933) * C * A @ I + I @ I @ I @ (6283185.307179586) * C * A + ((3141519237.6520114 * exp(1j * (5098851102170539.0 * t)) + 3141519237.6520114 * exp(1j * (-5098851102170539.0 * t))) * |0><3| + |3><0| + (3141592653.589793 * exp(1j * (5098851102170539.0 * t)) + 3141592653.589793 * exp(1j * (-5098851102170539.0 * t))) * |1><2| + |2><1|) @ I @ exp(1j * 0.0025563558694769676 * (A + C)) @ I + ((3141592653.589793 * exp(1j * (5098930540482378.0 * t)) + 3141592653.589793 * exp(1j * (-5098930540482378.0 * t))) * |0><3| + |3><0| + (3141666071.2432733 * exp(1j * (5098930540482378.0 * t)) + 3141666071.2432733 * exp(1j * (-5098930540482378.0 * t))) * |1><2| + |2><1|) @ I @ exp(1j * 0.0025563558694769676 * (A + C)) @ I
